In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

In [ ]:
from grain_cloud import GrainCloud
from audio_clip import AudioClip, clips_from_folder
from utils import get_files_of_types
import definitions
from signal_processing import processors
import plotly.express as px

from copy import deepcopy

In [ ]:
directory = "path/to/audio/files"
files = get_files_of_types(directory, definitions.AUDIO_FILE_TYPES)
clips = clips_from_folder(directory, random_subset=1000)

In [ ]:
# ws = 16384 * 4
ws = 8192*4
cloud = GrainCloud(deepcopy(clips[:]))
new_clip = cloud.render(
    clip_processors=[
        lambda x: processors.strip_silence(x, -30, 0), 
        processors.normalize
        ],
    master_processors=[
        # lambda x: processors.most_dissimilar_segments(x, 20, ws, ws, 0.1, crossfade_ratio=0.1), 
        lambda x: processors.sparsify_by_mfccs(x, 300, window_size=ws, hop_size=ws, crossfade_ratio=0.05),
        processors.normalize
        ],
    consolidate_processor=lambda x: processors.cross_fade_all(x, 10)
)

new_clip.display()

In [ ]:
new_clip.save()

In [ ]:
from analysis import sparsify_df

df_mfccs = cloud.compute_mfccs(16384, 16384, 20)

# Get a list of column names with the prefix 'mfcc'
mfcc_columns = [col for col in df_mfccs.columns if col.startswith('mfcc')]
df_sparse = sparsify_df(df_mfccs, 100, mfcc_columns)
df_sparse

In [ ]:
px.scatter_3d(df_mfccs_scaled, x='mfcc_0', y='mfcc_1', z='mfcc_2', opacity=0.8, color='title')

In [ ]:
px.scatter_3d(df_sparse, x='mfcc_0', y='mfcc_1', z='mfcc_2', opacity=0.8, color='title')

In [ ]:
# ws = 16384 * 4
ws = 8192*1
cloud = GrainCloud(deepcopy(clips[:]))
new_clip = cloud.render(
    clip_processors=[
        lambda x: processors.strip_silence(x, -30, 0), 
        # lambda x: processors.sparsify_by_mfccs(x, 3, window_size=ws, hop_size=ws, crossfade_ratio=0.1),
        # lambda x: processors.most_dissimilar_segments(x, 20, ws, ws, 0.5, crossfade_ratio=0.1), 
        processors.normalize
        ],
    master_processors=[
        # lambda x: processors.most_dissimilar_segments(x, 20, ws, ws, 0.1, crossfade_ratio=0.1),
        lambda x: processors.sparsify_by_mfccs(x, 14, window_size=ws, hop_size=ws, crossfade_ratio=0.1),
        processors.normalize
        ],
    consolidate_processor=lambda x: processors.cross_fade_all(x, 1000)
)

new_clip.display()

In [ ]:
from signal_processing.features import samples_mfcc_df

df_fig = samples_mfcc_df(new_clip.samples, ws, ws, 20)

px.scatter_3d(df_fig, x='mfcc_0', y='mfcc_1', z='mfcc_2', opacity=0.8)

In [ ]:
# ws = 16384 * 4
ws = 8192*1
cloud = GrainCloud(deepcopy(clips[:]))
new_clip = cloud.render(
    clip_processors=[
        lambda x: processors.strip_silence(x, -30, 0), 
        # lambda x: processors.most_dissimilar_segments(x, 20, ws, ws, 0.5, crossfade_ratio=0.1), 
        processors.normalize
        ],
    master_processors=[
        lambda x: processors.most_dissimilar_segments(x, 20, ws, ws, 0.03, crossfade_ratio=0.1),
        # lambda x: processors.sparsify_by_mfccs(x, 200, window_size=ws, hop_size=ws, crossfade_ratio=0.1),
        processors.normalize
        ],
    consolidate_processor=lambda x: processors.cross_fade_all(x, 1000)
)

new_clip.display()

In [ ]:
df_fig = samples_mfcc_df(new_clip.samples, 8192, 8192, 20)

px.scatter_3d(df_fig, x='mfcc_0', y='mfcc_1', z='mfcc_2', opacity=0.8)